In [1]:
import numpy as np
import pandas as pd
import dash
from jupyter_dash import JupyterDash
import plotly.express as px
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from matplotlib import pyplot as plt
import folium
import geopandas as gpd
import dash_html_components as html
import dash_core_components as dcc
import plotly.offline as pyo
import plotly.graph_objs as go
pd.set_option('display.max_columns', None)


/var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/ipykernel_37189/1523722841.py:12: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/ipykernel_37189/1523722841.py:13: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
df_ghg_country = pd.read_csv("Data/ghg-data.csv")
df_ghg_country['gdp_per_capita']=df_ghg_country['gdp']/df_ghg_country['population']
countries_gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
df_ghg_country = pd.merge(df_ghg_country, countries_gdf, left_on = 'iso_code', right_on = 'iso_a3')

FileNotFoundError: [Errno 2] No such file or directory: 'Data/ghg-data.csv'

In [ ]:
dftest = pd.read_excel('data.xls')

In [ ]:
df_gdp = pd.read_excel("Data/gdp_full.xls", sheet_name='Data')
df_gdp = pd.melt(df_gdp, id_vars = [ "Country Name", "Country Code"], value_vars=df_gdp.iloc[:,-62:])
df_gdp = df_gdp.rename(columns={"value":"GDP per capita (current US$)"})
df_gdp = df_gdp[df_gdp['variable']!='Unnamed: 66']
df_gdp=df_gdp.rename(columns = {"variable":"Year"})
df_gdp['Year']=df_gdp['Year'].astype(int)
df_gdp['GDP per capita (current US$)'] = df_gdp['GDP per capita (current US$)'].astype(float)

In [ ]:
df_ghg_country = pd.merge(df_ghg_country, df_gdp, left_on=("iso_code", "year"), right_on= ('Country Code','Year'))
df_ghg_country.sort_values('year', inplace=True)

In [ ]:
df_ghg_continent = (
    df_ghg_country.groupby(["continent", "year"], as_index=False)[
        ["co2_per_capita", "GDP per capita (current US$)"]
    ]
    .sum()
)

df_ghg_continent["country"] = df_ghg_continent["continent"]
df_ghg_continent["continent"] = "world"

df_ghg_world = (
    df_ghg_country.groupby([ "year"], as_index=False)
    [
        ["co2_per_capita", "GDP per capita (current US$)"]
    ]
    .sum()
)
df_ghg_world['country']='world'
df_ghg_world['continent']=''


df_ghg_total = df_ghg_continent.append(
            df_ghg_country[["year", "country","continent", "co2_per_capita", "GDP per capita (current US$)"]])
df_ghg_total = df_ghg_total.append(
df_ghg_world[["year", "country","continent", "co2_per_capita", "GDP per capita (current US$)"]])


mask = df_ghg_total.year == 2020
df_tmp = (
    df_ghg_total[mask]
    .dropna(subset="co2_per_capita")[["country", "continent", "co2_per_capita"]]
    .fillna("")
)

fig = go.Figure(
    go.Treemap(
        labels=df_tmp["country"],
        values=df_tmp["co2_per_capita"],
        parents=df_tmp["continent"],
        branchvalues='total',
        root_color="lightgrey",
        textinfo = "label+ percent parent",
        texttemplate='%{label} <br> %{value: text} <br> %{percentRoot}',
        pathbar_textfont_size=15,
    )
)
fig.show()

/var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/ipykernel_34027/3005098798.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/ny/x98qxh1124dbtz5g3f6s09hh0000gn/T/ipykernel_34027/3005098798.py:24: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [ ]:
type(sliders)

list

In [ ]:
df_ghg_total = df_ghg_total[df_ghg_total["year"] >= 1960]
year0 = min(df_ghg_total["year"])
years = np.arange(year0, 2022)

#Make a list of frame
frame0 = None
frames = []
for year in years:
    mask = df_ghg_total.year == year
    df_tmp = (
        df_ghg_total[mask]
        .dropna(subset="co2_per_capita")[["country", "continent", "co2_per_capita"]]
        .fillna("")
    )   
    treemap = go.Treemap(
        labels=df_tmp["country"], 
        values=df_tmp["co2_per_capita"],
        parents=df_tmp["continent"],
        branchvalues="total",
        pathbar_textfont_size=15,
        root_color="lightgrey",
        textinfo = "label+value+percent parent"
    )
    if frame0 is None:
        frame0 = treemap
    frames.append(go.Frame(name=f"frame-{year}", data=treemap
    ))
#Make sliders
sliders = [
    dict(
        steps=[
            dict(
                method="animate",
                args=[
                    [f"frame-{year}"],
                    dict(
                        mode="e", frame=dict(redraw=True), transition=dict(duration=200)
                    ),
                ],
                label=f"{year}",
            )
            for year in years
        ],
        transition=dict(duration=0),
        x=0,
        y=0,
        currentvalue=dict(
            font=dict(size=12), prefix="Year: ", visible=True, xanchor="center"
        ),
        len=1.0,
        active=1,
    )  
]
#create the layout object with slider parameters
layout = {
    "title": f"Who emits the most Co2 (Mtons)?",
    "xaxis": {"visible": True, "showline": True},
    "yaxis": {"type": "log", "visible": True, "showline": True},
    "updatemenus": [
        {
            "type": "buttons",
            "buttons": [
                {
                    "method": "animate",
                    "label": "play",
                    "args": [
                        None,
                        dict(
                            frame=dict(duration=600, redraw=True),
                            transition=dict(duration=200),
                            fromcurrent=True,
                            mode="immediate",
                        ),
                    ],
                }
            ],
        }
    ],
    "sliders": sliders,
}
#Create the final figure with layout and frame parameters
figure = go.Figure(
    data=frame0,
    layout=layout,
    frames=frames
)

figure.show()


In [ ]:
import chart_studio
import chart_studio.plotly as py
username = 'Lanchu' # your username
api_key = '7fGqX5WyWgUDb0vlyeHL' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
py.plot(figure, filename = 'who emits the most Co2', auto_open=True)

'https://plotly.com/~Lanchu/17/'